Before you submit this notebook, make sure everything runs as expected in the local test cases. 
Please, paste the solution to the designed cell and do not change anything else.

Also, please, leave your first and last names below

In [ ]:
FirstName = "Musa"
LastName = "Safin"

---

In [1]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from scipy.spatial.distance import cdist # You can use this for rbf kernel

import unittest
import sys
import io

In [2]:
def rbf(x_1, x_2, sigma=1.):
    '''Computes rbf kernel for batches of objects
    Args:
        x_1: torch.tensor shaped `(#samples_1, #features)` of type torch.float32
        x_2: torch.tensor shaped `(#samples_1, #features)` of type torch.float32
    Returns:
        kernel function values for all pairs of samples from x_1 and x_2
        torch.tensor of type torch.float32 shaped `(#samples_1, #samples_2)`
    '''
    distances = np.exp(-(np.square(x_2.numpy()).sum(axis=1) + np.transpose([np.square(x_1.numpy()).sum(axis=1)]) - 2*np.matmul(x_1.numpy(), np.transpose(x_2.numpy())))/(2*sigma**2))
    return torch.Tensor(distances).type(torch.float32)

def hinge_loss(scores, labels):
    '''Mean loss for batch of objects
    '''
    assert len(scores.shape) == 1
    assert len(labels.shape) == 1
    return torch.max(1.0 - scores*labels, torch.zeros(labels.shape)).mean()


class SVM(BaseEstimator, ClassifierMixin):
    @staticmethod
    def linear(x_1, x_2):
        '''Computes linear kernel for batches of objects
        
        Args:
            x_1: torch.tensor shaped `(#samples_1, #features)` of type torch.float32
            x_2: torch.tensor shaped `(#samples_1, #features)` of type torch.float32
        Returns:
            kernel function values for all pairs of samples from x_1 and x_2
            torch.tensor shaped `(#samples_1, #samples_2)` of type torch.float32
        '''
        return x_1 @ x_2.T
    
    def __init__(
        self,
        lr: float=1e-3,
        epochs: int=2,
        batch_size: int=64,
        lmbd: float=1e-4,
        kernel_function=None,
        verbose: bool=False,
    ):
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        self.lmbd = lmbd
        self.kernel_function = kernel_function or SVM.linear
        self.verbose = verbose
        self.fitted = False

    def __repr__(self):
        return 'SVM model, fitted: {self.fitted}'

    def fit(self, X, Y):
        assert (np.abs(Y) == 1).all()
        n_obj = len(X)
        X, Y = torch.FloatTensor(X), torch.FloatTensor(Y)
        K = self.kernel_function(X, X).float()

        self.betas = torch.full((n_obj, 1), fill_value=0.001, dtype=X.dtype, requires_grad=True)
        self.bias = torch.zeros(1, requires_grad=True) # I've also add bias to the model
        
        optimizer = optim.SGD((self.betas, self.bias), lr=self.lr)
        for epoch in range(self.epochs):
            perm = torch.randperm(n_obj)  # Generate a set of random numbers of length: sample size
            sum_loss = 0.                 # Loss for each epoch
            for i in range(0, n_obj, self.batch_size):
                batch_inds = perm[i:i + self.batch_size]
                x_batch = X[batch_inds]   # Pick random samples by iterating over random permutation
                y_batch = Y[batch_inds]   # Pick the correlating class
                k_batch = K[batch_inds]
                
                optimizer.zero_grad()     # Manually zero the gradient buffers of the optimizer
                
                preds = k_batch @ self.betas + self.bias # get the matrix product using SVM parameters: self.betas and self.bias
                preds = preds.flatten()
                loss = self.lmbd * self.betas[batch_inds].T @ k_batch @ self.betas + hinge_loss(preds, y_batch)
                loss.backward()           # Backpropagation
                optimizer.step()          # Optimize and adjust weights

                sum_loss += loss.item()   # Add the loss

            if self.verbose: print("Epoch " + str(epoch) + ", Loss: " + str(sum_loss / self.batch_size))

        self.X = X
        self.fitted = True
        return self

    def predict_scores(self, batch):
        with torch.no_grad():
            batch = torch.from_numpy(batch).float()
            K = self.kernel_function(batch, self.X)
            # compute the margin values for every object in the batch
            return (K @ self.betas + self.bias).flatten()

    def predict(self, batch):
        scores = self.predict_scores(batch)
        answers = np.full(len(batch), -1, dtype=np.int64)
        answers[scores > 0] = 1
        return answers

### Test 0: Initialization (0.01 points)

In [3]:
# do not change this cell
from sklearn.datasets import make_circles
X, y = make_circles(150, factor=.1, noise=.1, random_state=42)



### Test 1: SVM accuracy (0.69 points)

In [4]:
X_test, y_test = X[100:], y[100:]
X, y = X[:100], y[:100]
y[y==0] = -1 # for convenience with formulas
y_test[y_test==0] = -1
clf = SVM(epochs=150, lr=0.1, batch_size=20, verbose=False, kernel_function=rbf)
clf.fit(X, y)
pred = clf.predict(X_test)
assert accuracy_score(y_test, pred) > 0.95

### Test 2: Kernel (0.3 points)

In [5]:
assert (np.allclose(rbf(X_kernel, X_kernel).detach().cpu().numpy(), np.array([[1.0000e+00, 2.2897e-11],[2.2897e-11, 1.0000e+00]])))

NameError: ignored